# Conv3d Refinements

In [1]:
import os
import cv2
import numpy as np
import datetime as dt
import shutil
import pandas as pd
import random
from itertools import product
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import colormaps

import tensorflow as tf

from keras.layers import *
from keras.models import Sequential
from keras.utils import plot_model, to_categorical
from keras.models import load_model
from keras.optimizers import Adam

from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay, classification_report, confusion_matrix, accuracy_score


In [146]:
# List files and ignore .DS_Store if on a Mac
def list_files(directory):
    visible_files = []
    for file in os.listdir(directory):
        if not file.startswith('.'):
            visible_files.append(file)

    return visible_files


def video_to_frames(video_path, img_size, sequence_length):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, img_size)
        frames.append(frame)
        if len(frames) == sequence_length:
            break
    cap.release()

    if len(frames) < sequence_length:
        return None  # Ignore short videos

    return np.array(frames)


# 1. Load and Preprocess Videos
def load_videos_from_folders(folder_path, img_size=(64, 64), sequence_length=30):
    # classes = os.listdir(folder_path)
    classes = list_files(folder_path)
    data, labels = [], []

    for label, activity in enumerate(classes):
        activity_folder = os.path.join(folder_path, activity)
        # for video_file in os.listdir(activity_folder):
        for video_file in list_files(activity_folder):
            video_path = os.path.join(activity_folder, video_file)
            frames = video_to_frames(video_path, img_size, sequence_length)
            if frames is not None:
                data.append(frames)
                labels.append(label)

    data = np.array(data)
    labels = to_categorical(labels, num_classes=len(classes))

    return data, labels, classes

def get_classes(directory):
    # Get Names of all classes
    classes = list_files(directory)
    # classes = enumerate(classes)

    return classes

## Access data, labels, and classes in the dataset

In [147]:
test_data_folder = "../downloads/r2_test/"
data, labels, classes = load_videos_from_folders(test_data_folder)

In [148]:
# Load model
model_path = './2024-11-05-02-15-08-conv3d-model.keras'
model = load_model(model_path)

In [149]:
def make_predictions3d(model, idx):
    # Interpreting the data structure
    predict_idx = idx
    selected_data = data[predict_idx]
    selected_labels = labels[predict_idx]
    true_label = np.argmax(selected_labels, axis=0)
    true_class = classes[true_label]

    # Add extra dimension
    reshaped_data = tf.expand_dims(selected_data, axis=0)

    test_predictions = model.predict(reshaped_data)

    predicted_label = np.argmax(test_predictions, axis=1)
    predicted_class = classes[predicted_label[0]]

    print(f'Predicted class: {predicted_class}\tTrue class: {true_class}')

    if predicted_label == true_label:
        correct = True
    else:
        correct = False

    return predicted_label, predicted_class, true_label, true_class, correct


In [154]:
# Predict one file
directory = "../downloads/r2_test"
classes = get_classes(directory)

classname = "Family visit"
file = "7393781501487533201_r1.mp4"
video_path = f"../downloads/r2_test/{classname}/{file}"

frames = video_to_frames(video_path, img_size=(64, 64), sequence_length=30)
frames = np.array([frames])
test_predictions = model.predict(frames)
predicted_label = np.argmax(test_predictions, axis=1)
predicted_label = predicted_label[0]
print(f'Predicted label: {predicted_label}\tPredicted class: {classes[predicted_label]}\tTrue class: {classes[0]} ')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Predicted label: 9	Predicted class: Watching TV	True class: Eating 


# Predict entire test set

In [86]:
# Track correct predictions
correct_predictions = 0
incorrect_predictions = 0

idx = 0

predicted_labels = []
predicted_classes = []
true_labels = []
true_classes = []

# Iterate though test directory
while idx < len(data):

    pl, pc, tl, tc, correct = make_predictions3d(model, idx)

    predicted_labels.append(pl)
    predicted_classes.append(pc)
    true_labels.append(tl)
    true_classes.append(tc)

    if correct:
        correct_predictions += 1
    else:
        incorrect_predictions += 1

    idx += 1

print(f'Correct: {correct_predictions}\tIncorrect: {incorrect_predictions}')

# Save data
results_df = pd.DataFrame({"Predicted label": predicted_labels, 
                           "Predicted class": predicted_classes,
                           "True label": true_labels, 
                           "True class": true_classes})

date_time_format = '%Y-%m-%d-%H-%M-%S'
current_date_time_dt = dt.datetime.now()
current_date_time_string = dt.datetime.strftime(current_date_time_dt, date_time_format)

file = f'../data/{current_date_time_string}-conv3d-testing.csv'

results_df.to_csv(file, index=False) 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Predicted class: Eating	True class: Eating
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Predicted class: Eating	True class: Eating
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Predicted class: Eating	True class: Eating
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Predicted class: Eating	True class: Eating
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Predicted class: Eating	True class: Eating
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Predicted class: Eating	True class: Eating
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Predicted class: Eating	True class: Eating
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Predicted class: Eating	True class: Eating
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Predicted class: Eating	True class: Eating
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Predicted class: Eating	True class: Eating
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Predicted class: Eating	True class: Eating
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Predicted class: Eating	True class: Eating
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 